# Hacettepe University Computer Science Department
## Assignment 1
***

Name and Surname : _Burak Emre Ozer_

Identity Number : _21527266_

Course : _Introduction to Machine Learning Lab. (BBM409)_

Due : _06/11/2018_

Advisors: _Dr. Aykut Erdem, T.A. Necva Bölücü_



## PART 1: Theory Questions

***

##### k-Nearest Neighbor Classification

#### 1. Let k-NN(S) be the k Nearest Neighbor classification algorithm on sample set S, which takes the majority of the closest k points where there are 2 classes (positive and negative).
* _p1 ∈ S1, p1 is the closest point of x which is positive. p2 ∈ S2, p2 is the closest point of x which is positive. Then 1-kNN(S1US2) = p where p=p1 or p=p2 which is p is positive._

<img src="images/1.jpeg">

* _Show an example such that in both 3-NN(S 1 ) and 3-NN(S 2 ) the label of x is positive, and in 3-NN(S 1 ∪ S 2 ) the label of x is negative._
<img src="images/2.jpeg">

#### 2.  One of the problems with k-nearest neighbor learning is selecting a value for k. Say you are given the following data set. This is a binary classification task in which the instances are described by two real-valued attributes.

* k = 0 minimizes the training error. The error rate is 0.
* k = 5 or k = 7 minimizes the LOOCV error. The error rate is 2/7.
* Too small k values leads to overfitting. Too big k values misclassifies every datapoint.
* <img src="images/3.png">

##### Linear Regression

### 1.
X is 23×6, y is 23×1, θ is 6×1.

### 2. 
Gradient descent, since (XTX)−1 will be very slow to compute in the normal equation. With n = 200000 features, we have to invert a 200001 x 200001 matrix to compute the normal equation. Inverting such a large matrix is computationally expensive, **so gradient descent is a good choice.**

### 3. 

* (a) _**False**_ The magnitude of the feature values are insignificant in terms of computational cost.
* (b) _**False**_
* (c) _**False**_ The cost function J(θ) for linear regression has no local optima.
* (d) _**True**_ Feature scaling speeds up gradient descent by avoiding many extra iterations that are required when one or more features take on much larger values than the rest.

## PART 2: Book Recommendation System
***

## 1.Aim

The main goal in this assignment is building a system that predicting a rate for the book based on that the user’s preference and liking using the average rating or weighted average rating of top-k nearest neighbor.

### 1.1 Collaborative Filtering

Collaborative Filtering (CF) is a mean of recommendation based on users’ past behavior. There are two categories of CF:

**User-based:** measure the similarity between target users and other users

**Item-based:** measure the similarity between the items that target users rates/ interacts with and other items

The key idea behind CF is that similar users share the same interest and that similar items are liked by a user.In this assignment, I used User-based CF by the way.

### 1.1.1 User-based Collaborative Filtering

We need to compute the similarity between users in user-based CF. There are two options, Pearson Correlation or cosine similarity.

### 1.2 Similarities

#### Pearson Product-Moment Correlation

Pearson product-moment correlation attempts to draw a line of best fit through the data of two variables, and the Pearson correlation coefficient, r, indicates how far away all these data points are to this line of best fit.

I chose the Pearson Correlation of these options because it had a lower error value. 

The difference is that Pearson Correlation is invariant to adding a constant to all elements.


_Pearson Correlation:_ |  _Predicted value_
:-------------------------:|:-------------------------:
<img src="images/4.png"> | <img src="images/5.png">



## 2. Data source and cleaning the data
***
###  _2.1 Dataset_

Book-Crossings is a book rating dataset compiled by Cai-Nicolas Ziegler. The data we are going to use is from [here](http://www2.informatik.uni-freiburg.de/~cziegler/BX/).  

The Book-Crossing dataset comprises 3 tables.

* **BX-Users**

&ensp;&ensp;&ensp;&ensp;Contains the users. Note that user IDs (`User-ID`) have been anonymized and map to integers. Demographic data is provided (`Location`, `Age`) if available. Otherwise, these fields contain NULL-values.

* **BX-Books**

&ensp;&ensp;&ensp;&ensp;Books are identified by their respective ISBN.Moreover, some content-based information is given (`Book-Title`, `Book-Author`, `Year-Of-Publication`, `Publisher`).

* **BX-Book-Ratings**

&ensp;&ensp;&ensp;&ensp;Contains the book rating information. Ratings (`Book-Rating`) are either **explicit**, expressed on a scale from **1-10** (higher values denoting higher appreciation), or implicit, expressed by 0.

###  _2.2 Cleaning the data_

In order to improve computing speed, and
not run into the ’MemoryError’ issue, I filter the user data to those in the US or Canada.

And the original data contains ratings of value 0, which indicate implicit feedback. Since I will only use explicit feedbacks, first filter out the ratings that are between 1 to 10. 

&ensp;&ensp;&ensp;&ensp;`us_canada_user_rate = us_canada_user_rate[us_canada_user_rate['bookRating'] > 0]`


With implicit data             | Without implicit data
:-------------------------:|:-------------------------:
<img src="images/6.png"> | <img src="images/7.png">


Also the dataset is very sparse and I filtered the dataset with items that were rated **at least 1 times** and users who have rated **at least 1 items**. At this stage, the dataset contain 25799 ratings, 8949 users and 14480 items. 

&ensp;&ensp;&ensp;&ensp;`us_canada_user_rate.groupby('ISBN').filter(lambda x: len(x) >= 1)`

&ensp;&ensp;&ensp;&ensp;`us_canada_user_rate.groupby('userID').filter(lambda x: len(x) >= 1)`

## 3. Steps to follow
***

1. _Cleaning and combined the data as above._

***

2. _Storing the data into a dataframe using the pandas. Then I've convert it a pivot table then dictionary to easily use._

&ensp;&ensp;&ensp;&ensp;`user_rating_pivot = train.pivot_table(index='ISBN', columns='userID', values='bookRating', fill_value=0)`

&ensp;&ensp;&ensp;&ensp;`user_rating_pivot.to_dict()`

***

3. _Calculating the similarities between rating verctors then find the k-nearest neighbors._

<pre><code>
def pearson(target_user_rates, other_user_rates):
    user1 = list(target_user_rates.values())
    user2 = list(other_user_rates.values())
    return np.corrcoef(user1, user2)[0][1]
</code></pre>

***


4. _Now, we can predict the users’ rate on the unrated items with **kNN** or the below equation which is **weighted kNN** where the weight is 1/manhattan distance or pearson correlation._

<pre><code>
    for user in sorted_x[:l]:
        other_uid = user[0]
        weight = user[1]
        rate = ubr_dict[other_uid][isbn_id]
        part += rate * weight
        dom += weight
    return part/dom
</code></pre> 


# 4. Analysis of the results for prediction
***

### 4.1 Validation set

<img src="images/8.png">

* Randomly divide the available set of samples into two parts: a training set and a validation or hold-out set.
* The model is fit on the training set, and the fitted model is used to predict the responses for the observations in the validation set.
* The resulting validation-set error provides an estimate of the test error. 

Also based on the validation data, I found that the most suitable model is in accordance with k = 11 with weighted kNN then I tested model it's MAE value 1.36 which is quite successful.

### 4.2 Comparison between weighted-kNN and kNN

Weighted KNN implements a fixed combiner rule. In contrast to this approach, KNN a majority voting rule. 

Weighted kNN             |kNN
:-------------------------:|:-------------------------:
<img src="images/9.png"> | <img src="images/10.png">



So when I analyzed the results, I saw that weighted knn gives more accurate results. Optimal k value
is 11.

### 4.3 Data filtering and effects

I filtered the dataset with items that were rated at least 1,2,3,4,5 times and users who have rated at least 1,2,3,4,5 items. In this way I increased the success rate.

<img src="images/11.png">

### 4.4 About the Cold Start Problem

Collaborative filtering systems generate recommendation based on user user similarity. 

A new user encounters a serious problem in the collaborative filtering approach. Since the system does not have any data about the new user preferences, it could not provide any personalized recommendation for him/her. 

#### How did I approach this situation?

I predict the average rates given to the book to solve this situation for the new user. I also predict the average of the user when a new book was added.

<pre><code>
def find_book_avg(isbn_id):
    book_list = user_rating_pivot.loc[isbn_id].tolist()
    book_votes = [vote if vote != 0 else np.nan for vote in book_list]
    book_avg = np.nanmean(np.array(book_votes))
    return book_avg


def find_user_avg(uid):
    vote_list = _dict[uid].values()
    vote_list = [vote if vote != 0 else np.nan for vote in vote_list]
    vote_avg = np.nanmean(np.array(vote_list))
    return vote_avg
</code></pre>


### 4.5 Conclusion

In the k-nearest neighbor approach to user-based collaborative filtering I use k most similar people to predict ratings. And also, the most difficult issue was that the data too sparse. I tried very hard to clean the data. After obtaining clean data, the results were satisfactory. 



## Resources

https://towardsdatascience.com/collaborative-filtering-based-recommendation-systems-exemplified-ecbffe1c20b1

https://medium.com/@cfpinela/recommender-systems-user-based-and-item-based-collaborative-filtering-5d5f375a127f

https://github.com/bubabi/foodmates/blob/master/common/recommender.py

http://www.diva-portal.org/smash/get/diva2:1111865/FULLTEXT01.pdf

https://www.researchgate.net/figure/Example-user-based-collaborative-filtering_fig4_220116257